### Prepare the Json file a git from the Code in VS:

In [1]:
import json
import os
from PIL import Image
import io

In [ ]:
def Get_Image(Image_name):
    filename = Image_name
    image_path = os.path.join("PATH_YOUR_TRAINING_IMAGES",filename)
    print(image_path)
    with open(image_path, 'rb') as file:
        binary_image_data = file.read()
    image = Image.open(io.BytesIO(binary_image_data))
    return image

def Get_Annoutaions(Annoutaion):
    bboxes = []
    ner_tags = []
    tokens = []
    Number_annotaions = len(Annoutaion)
    for j in range(Number_annotaions):
        box = Annoutaion[j]["box"]
        text = Annoutaion[j]["text"]
        tag = Annoutaion[j]["label"]
        #Transform box data :
        box = [int(x * 10) for x in box]
        # Append
        bboxes.append(box)
        tokens.append(text)
        ner_tags.append(tag)

    return bboxes,tokens,ner_tags


def Prepare_Data(Json_path):
    dataSet = []
    # Read the JSON file
    with open(Json_path, 'r') as file:
        data = json.load(file)
    n = len(data) # n : Number_Images
    for i in range(n):
          ###########################################
          image = Get_Image(data[i]["file_name"])
          ########################################
          bboxes,tokens,ner_tags=Get_Annoutaions(data[i]["annotations"])
          #print(ner_tags)
          ############################################
          document_dict = {
              'id': i,
              'image': image,
              'bboxes': bboxes,
              'ner_tags': ner_tags,
              'tokens': tokens
          }
          dataSet.append(document_dict)
    return dataSet



In [ ]:
Json_path = "PATH_YOUR_TRAINING_JSON_FILE"
dataSet = Prepare_Data(Json_path)
print("Number of Images : ",len(dataSet))

/content/drive/MyDrive/Project_AM/Training/Data/104.jpg
/content/drive/MyDrive/Project_AM/Training/Data/114.jpg
/content/drive/MyDrive/Project_AM/Training/Data/12.jpg
/content/drive/MyDrive/Project_AM/Training/Data/129.jpg
/content/drive/MyDrive/Project_AM/Training/Data/13.jpg
/content/drive/MyDrive/Project_AM/Training/Data/131.jpg
/content/drive/MyDrive/Project_AM/Training/Data/135.jpg
/content/drive/MyDrive/Project_AM/Training/Data/14.jpg
/content/drive/MyDrive/Project_AM/Training/Data/143.jpg
/content/drive/MyDrive/Project_AM/Training/Data/150.jpg
/content/drive/MyDrive/Project_AM/Training/Data/151.jpg
/content/drive/MyDrive/Project_AM/Training/Data/172.jpg
/content/drive/MyDrive/Project_AM/Training/Data/184.jpg
/content/drive/MyDrive/Project_AM/Training/Data/187.jpg
/content/drive/MyDrive/Project_AM/Training/Data/195.jpg
/content/drive/MyDrive/Project_AM/Training/Data/197.jpg
/content/drive/MyDrive/Project_AM/Training/Data/2.jpg
/content/drive/MyDrive/Project_AM/Training/Data/202.j

In [ ]:
dataSet[0]

### Libraries :

In [ ]:
!pip install -q datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

### Log In

In [ ]:
from huggingface_hub import notebook_login
# hf_XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX , this the token to put , Get Yours
notebook_login()

### Create A Hugging Face DataSet
- If you already host the dataSet in Hugging you don't need to this step.
- If you run this again you will ouverwrite the old dataSet in Hugging.
- To push the dataSet to Hugging you should already login with your Token.
- make shure to make your data Private (optional), but if you want to visualize the dataSet in Hugging you will need to make it public.

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, Features, Sequence, ClassLabel, Value, Image

In [ ]:
dataset_features = Features({
    'id': Value('string'),
    'image': Image(decode=True),
    'bboxes': Sequence(Sequence(Value('int64'))),
    'ner_tags': Sequence(ClassLabel(names=['InvNum','InvDate', 'Fourni' ,'TTC','TVA','TT' ,'Autre'])),
    'tokens': Sequence(Value('string')),
})

In [ ]:
# Convert the final_dataset into a dictionary of lists
data_dict = {
    'id': [item['id'] for item in dataSet],
    'image': [item['image'] for item in dataSet],
    'bboxes': [item['bboxes'] for item in dataSet],
    'ner_tags': [item['ner_tags'] for item in dataSet],
    'tokens': [item['tokens'] for item in dataSet],
}

In [ ]:
hf_dataset = Dataset.from_dict(data_dict, features=dataset_features)
ds = hf_dataset.train_test_split(test_size=0.07,shuffle=True,seed =10)

In [ ]:
ds.push_to_hub("ID_YOUR_DATASET_NAME")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Noureddinesa/LayoutLmv3_v1/commit/5f7ce9edac8377bbf3b5421e49f801b1d33ae929', commit_message='Upload dataset', commit_description='', oid='5f7ce9edac8377bbf3b5421e49f801b1d33ae929', pr_url=None, pr_revision=None, pr_num=None)